In [29]:
import numpy as np
np.set_printoptions(precision=2)

room_temp = 18.333
side_length = 11
pan_temp = room_temp
oven_temp = 162.778
brownie_target = 85

h = 9.14
A = 0.0001 # approx 0.01m ^2, 5.208 degree slice

brownie_density = 500
brownie_specific_heat = 2550
brownie_conductivity = 0.15 * 3
brownie_alpha = brownie_conductivity / (brownie_density * brownie_specific_heat)

steel_density = 7832
steel_specific_heat = 434
steel_conductivity = 59
steel_alpha = steel_conductivity / (steel_density * steel_specific_heat)

print(brownie_alpha)
print(steel_alpha)

def initialize_brownie_pan(side_length, room_temp):
    return np.full(side_length, room_temp)


3.529411764705882e-07
1.7357597096632978e-05


In [30]:
brownie_pan = initialize_brownie_pan(side_length, room_temp)

def heat_transfer_over_increment(brownie_pan: np.array, pan_temp, dr, dt):
    # oven to pan
    d_pan_temp = h * A * (oven_temp - pan_temp)
    pan_temp += dt * d_pan_temp 

    # pan to brownie
    time_derivative = np.zeros(brownie_pan.shape)
    for i, temp in enumerate(brownie_pan):
        if i == 0:
            outside_neighbour = brownie_pan[i+1]
            inside_neighbour = temp
        elif i == side_length - 1:
            outside_neighbour = pan_temp
            inside_neighbour = brownie_pan[i-1]
        else:
            outside_neighbour = brownie_pan[i-1]
            inside_neighbour = brownie_pan[i+1]

        r = i + 0.5
        first_term = (r-.5) * ((inside_neighbour - temp) / dr) - ((r+0.5) * (temp - outside_neighbour) / dr)
        first_term += (pan_temp * r)/dr
        time_derivative[i] = brownie_alpha * (1/r) * first_term * (1/dr)

    brownie_pan += (time_derivative * dt)
    
    return brownie_pan, pan_temp

In [31]:
brownie_pan = initialize_brownie_pan(side_length, room_temp)
counter = 0
while True:
    brownie_pan, pan_temp = heat_transfer_over_increment(brownie_pan, pan_temp, dr=0.02, dt=0.01) # 0.01 meters (1cm), 0.01 seconds
    counter += 1
    # if counter % 10000 == 0:
      # print(brownie_pan)
    if brownie_pan[0] >= brownie_target:

        print(f"{counter / 100 / 60} minutes of cooking")
        break

print(brownie_pan)

17.831333333333337 minutes of cooking
[85.   85.   85.   85.   85.   85.   85.02 85.15 85.85 88.97 98.4 ]


In [32]:
import math

cooked_baseline = np.full(side_length, brownie_target + 3)

over_cooked = brownie_pan - cooked_baseline
over_cooked = over_cooked.clip(min=0)
# avg_error = (np.sum(over_cooked) * (22 * math.pi)) / (math.pi * side_length**2)

avg_error = np.sum(over_cooked) / 11

print(avg_error)

1.033166866297293
